# We would like to confirm that more of their resources are dedicated to their customer facing products as the day goes on

### To do this we will have to compare total resource use, by total resources dedicated to their services

It is important for us to take care when adding these that we calculate resources correctly. We will want to look at something like this for every timestamp, for every server, for ever service instance, calculate total resource Util.

In [1]:
import pandas as pd
import numpy as np

In [2]:
service_containers = pd.read_csv("/home/codyblakeney/private/personal_repo/gc_project/data/container_event.csv")

In [3]:
service_instances = pd.read_csv("/home/codyblakeney/private/personal_repo/gc_project/data/container_usage.csv")

In [4]:
ids_at_39600 = np.array(service_instances[service_instances.ts == 39600].instance_id)

In [5]:
service_containers[service_containers.instance_id == ids_at_39600[0]]

,ts,event,instance_id,machine_id,plan_cpu,plan_mem,plan_disk,cpuset,Unnamed: 8
1428,0,Create,117,823,4,0.042409,0.034085,40|41|42|43,NaN


In [6]:
service_instances[np.logical_and(service_instances.instance_id == ids_at_39600[0] , service_instances.ts==39600)]

,ts,instance_id,cpu_util,mem_util,disk_util,load1,load5,load15,avg_cpi,avg_mpki,max_cpi,max_mpki
11,39600,117,3.66,38.299999,12.8,0.14,0.18,0.2,0.164531,0.227815,2.046043,3.524049


https://serverfault.com/questions/667078/high-cpu-utilization-but-low-load-average

### Miss match of CPU Util and Linux Load could mean VMs competing for resources

In [7]:
from bokeh.plotting import figure, show
from bokeh.charts import Histogram
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [8]:
p = Histogram(service_instances[service_instances.ts == 39600].load1, bins=15)
show(p)

In [9]:
len(service_instances[service_instances.ts == 39600].load1)

10260

In [10]:
from service_machine import *

## time_stamp_machine returns a dictionary schema is as follows

```
    { 
        machine_id: 
        {
            instance_id: 
            {
                plan_cpu:
                plan_mem:
                plan_disk:
                cpu_set: []
                cpu_util:
                mem_util:
                disk_util:
                load1:
                load5:
             }
         }
     }
```
        

In [12]:
machine_services = time_stamp_machine(39600, service_containers, service_instances)

In [19]:
cpus = []
for instance in machine_services[1]:
    for cpu in machine_services[1][instance]["cpu_set"]:
        cpus.append(int(cpu))
        
cpus.sort()

### As you might expect every CPU is being used. It doesn't appear that there is any overlap on this particular machine

In [21]:
cpus

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63]